In [259]:
%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats

import mne
from mne.channels import find_ch_adjacency
from mne.stats import bonferroni_correction, fdr_correction
from mne.stats import permutation_cluster_1samp_test, permutation_cluster_test

from utils import bad_participant


In [3]:
epoch_type = 'evoked'
# epoch_type = 'pseudo-rs'


all_participants = ['VP07','VP08','VP09', 'VP10','VP11','VP12','VP13','VP14','VP18','VP19','VP20','VP22','VP23','VP24','VP25','VP26','VP27','VP28','VP29','VP30','VP31','VP32','VP33','VP35','VP36','VP37']


path = '/media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/' #icm-linux
# path = '/Users/nicobruno/ownCloud/MW_eeg_data/minmarker/' #nico-mac

In [6]:
bad_participants = []
for i in all_participants:
    participant = i
    
    folder = path + participant +'/'
    epochs = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'ar_subtracted_epo.fif', verbose = False) 

    if bad_participant(epochs, probe= 'PC', mind = 'mind2'):
        bad_participants.append(participant)

print('List of bad participants: ', bad_participants)

good_participants = [x for x in all_participants if x not in bad_participants]
print(f'{len(good_participants)} good participants remaining')

['VP08', 'VP09', 'VP12', 'VP13', 'VP14', 'VP19', 'VP20', 'VP23', 'VP25', 'VP29', 'VP30', 'VP31', 'VP32', 'VP33']
12 participants remaining


In [18]:
n = 1
epochs[f'PC/dMW/correct/go/{n}']

<EpochsFIF |  2 events (all good), -0.2 - 0.6 sec, baseline [-0.2, 0] sec, ~296 kB, data loaded,
 'PC/dMW/go/correct/1/s001.0': 1
 'PC/dMW/go/correct/1/s002.0': 1>

In [271]:
evoked_mw = []
evoked_ot = []

for i in good_participants:
    participant = i

    folder = path + participant +'/'
    
    #############################
    #### With ERP SUBTRACTED ####
    #############################
    epochs = mne.read_epochs(folder +  participant + '_' + epoch_type + '_' +  'ar_rereferenced_epo.fif', verbose = False) 
    epochs.info['description'] = 'biosemi/64' #necessary for wSMI 
    epochs =  epochs.pick_types(eeg = True) #EOGs break everything\
    

    evoked_ot.append(epochs['PC/on-task/correct/go'].average().data)
    try:
        evoked_mw.append(mne.concatenate_epochs([epochs['PC/sMW/correct/go'], epochs['PC/dMW/correct/go']]).average().data)
    except:
        try:
            evoked_mw.append(epochs['PC/sMW/correct/go'].average().data)
        except:
            try:
                evoked_mw.append(epochs['PC/dMW/correct/go'].average().data)
            except:
                pass

In [116]:
ot_grand_average = mne.grand_average(evoked_ot)
mw_grand_average = mne.grand_average(evoked_mw)

Identifying common channels ...
Identifying common channels ...


In [222]:
a = mne.combine_evoked(evoked_ot, weights = 'equal')
a.data.shape

(64, 201)

In [177]:
contrasts = [ ot_grand_average,mw_grand_average]
contrast  = mne.combine_evoked(contrasts, weights = [1,-1])

In [192]:
channels = ['FC1', 'FCz', 'FC2', 'C1', 'Cz','C2', 'CP1', 'CPz', 'CP2']
# channels = ['Cz']
idx =  [contrast.ch_names.index(channel) for channel in channels]


In [195]:
mne.viz.plot_compare_evokeds(contrasts, picks = idx)
mne.viz.plot_compare_evokeds(contrast,idx)

TypeError: plot_compare_evokeds() got an unexpected keyword argument 'method'

In [277]:
tmax = 0.6


fig, ax = plt.subplots(1, figsize=(3.3, 2.3))
scale = 1e6
ax.plot(mw_grand_average.times * 1000, contrasts[0].data[idx][0] * scale,
        'r', label='MW')
ax.plot(ot_grand_average.times * 1000, contrasts[1].data[idx][0] * scale,
        'g', label='OT')
ax.plot(ot_grand_average.times * 1000, (contrasts[0].data[idx][0] * scale - contrasts[1].data[idx][0] * scale),
        'b', label='OT')

ax.grid(True)
ax.set(xlim=[-100, 1000 * tmax], xlabel='Time (in ms after stimulus onset)',
#        ylim=[-12.5, 5],
       ylabel=u'Potential difference (μV)')
ax.axvline(800, ls='--', color='k')

fig.tight_layout(pad=0.5)

plt.show()

In [278]:
np.array(evoked_mw).shape

(12, 64, 201)

In [279]:
T, pval = stats.ttest_rel(np.array(evoked_mw),np.array(evoked_ot), axis = 2)
pval

array([[1.57804848e-01, 4.70833723e-01, 1.59968440e-01, 6.23559758e-01,
        1.47009916e-04, 3.23777348e-01, 2.79180976e-01, 2.13638755e-28,
        2.70077788e-21, 3.11102161e-01, 9.63400298e-06, 1.04124003e-01,
        1.37753481e-31, 3.60913520e-26, 4.95363697e-10, 4.58011918e-12,
        1.39491373e-34, 2.07914573e-25, 2.35261162e-06, 6.51835496e-01,
        1.07780786e-15, 5.83921094e-24, 8.83282473e-09, 8.90724113e-01,
        3.11619976e-02, 2.81554649e-07, 7.81054909e-05, 1.81798814e-07,
        3.41812698e-03, 2.13984051e-09, 3.38319802e-11, 9.69331749e-01,
        6.90508914e-04, 4.37822393e-17, 2.99683591e-10, 3.80199378e-04,
        4.74802945e-09, 6.17782531e-03, 1.39910558e-05, 1.92701900e-03,
        2.45519969e-16, 2.88254681e-03, 2.00135008e-06, 1.00832838e-02,
        3.66893357e-08, 1.21523467e-01, 2.45827974e-03, 1.36560733e-04,
        1.55386508e-16, 1.23052598e-03, 4.21868945e-01, 1.23792960e-06,
        1.65515400e-09, 1.60143557e-02, 1.89635486e-10, 9.570700

In [281]:
T, pval = stats.ttest_rel(np.array(evoked_mw),np.array(evoked_ot), axis  =2)
alpha = 0.05

n_tests,  n_samples, ts = np.array(evoked_mw).shape
threshold_uncorrected = stats.t.ppf(1.0 - alpha, n_samples - 1)

reject_bonferroni, pval_bonferroni = bonferroni_correction(pval, alpha=alpha)
threshold_bonferroni = stats.t.ppf(1.0 - alpha / n_tests, n_samples - 1)

reject_fdr, pval_fdr = fdr_correction(pval, alpha=alpha, method='indep')
threshold_fdr = np.min(np.abs(T)[reject_fdr])

In [282]:
times = 1e3 * mw_grand_average.times

plt.close('all')
plt.plot(times, T, 'k', label='T-stat')
xmin, xmax = plt.xlim()
plt.hlines(threshold_uncorrected, xmin, xmax, linestyle='--', colors='k',
           label='p=0.05 (uncorrected)', linewidth=2)
plt.hlines(threshold_bonferroni, xmin, xmax, linestyle='--', colors='r',
           label='p=0.05 (Bonferroni)', linewidth=2)
# plt.hlines(threshold_fdr, xmin, xmax, linestyle='--', colors='b',
#            label='p=0.05 (FDR)', linewidth=2)
plt.legend()
plt.xlabel("Time (ms)")
plt.ylabel("T-stat")
plt.show()

ValueError: x and y must have same first dimension, but have shapes (201,) and (12, 64)

In [ ]:
n_permutations = 1000
T0, p_values, H0 = permutation_t_test(data, n_permutations, n_jobs=1)

In [268]:
data = [np.array(evoked_mw), np.array(evoked_mw)]
# adjacency, _ = find_ch_adjacency(epochs.info, "eeg")
n_permutations = 1000  # number of permutations to run

# set initial threshold
p_initial = 0.001

# set family-wise p-value
p_thresh = 0.5

connectivity = None

# set cluster threshold
n_samples = len(data)
# threshold = -stats.t.ppf(p_initial / (1 + (tail == 0)), n_samples - 1)
# threshold = 6


cluster_stats = permutation_cluster_test(
    data, threshold=threshold, verbose=True, tail=0,
#     adjacency =adjacency,
    n_permutations=n_permutations, seed=42)

T_obs, clusters, cluster_p_values, _ = cluster_stats

stat_fun(H1): min=0.000000 max=0.000000
Running initial clustering
Found 0 clusters


<ipython-input-268-f4aaf36d6213>:23: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  n_permutations=n_permutations, seed=42)


In [247]:
times = mw_grand_average.times
plt.close('all')
plt.subplot(211)
plt.title('Channel : ' + channel)
plt.plot(times, contrasts[0].data.mean(axis=0) - contrasts[0].data.mean(axis=0),
         label="ERF Contrast (Event 1 - Event 2)")
plt.ylabel("EEG ")
plt.legend()
plt.subplot(212)
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_p_values[i_c] <= 0.05:
        h = plt.axvspan(times[c.start], times[c.stop - 1],
                        color='r', alpha=0.3)
    else:
        plt.axvspan(times[c.start], times[c.stop - 1], color=(0.3, 0.3, 0.3),
                    alpha=0.3)
hf = plt.plot(times, T_obs, 'g')
plt.legend((h, ), ('cluster p-value < 0.05', ))
plt.xlabel("time (ms)")
plt.ylabel("f-values")
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'start'

In [249]:
times = 1e3 * contrast.times

fig, axes = plt.subplots(2, sharex=True, figsize=(3.3, 2.5))
ax = axes[0]
ax.plot(times, 1e6 * data.mean(axis=0), label="ERP Contrast")
ax.set(title='Channel : ' + channel, ylabel="EEG (uV)", ylim=[-5, 2.5])
ax.legend()
ax.annotate('A', (-0.16, 1.15))

ax = axes[1]
for i_c, c in enumerate(clusters):
    c = c[0]
    if cluster_p_values[i_c] < p_thresh:
        h1 = ax.axvspan(times[c.start], times[c.stop - 1],
                        color='r', alpha=0.3)
        
hf = ax.plot(times, T_obs, 'g')
ax.legend((h1,), (u'p < %s' % p_thresh,), loc='upper right', ncol=1)
ax.set(xlabel="time (ms)", ylabel="T-values",
       ylim=[-10., 10.], xlim=contrast.times[[0, -1]] * 1000)
fig.tight_layout(pad=0.5)
fig.savefig(op.join('..', 'figures', 'sensorstat_highpass-%sHz.pdf'
                    % (l_freq,)), bbox_to_inches='tight')
plt.show()

ValueError: x and y must have same first dimension, but have shapes (201,) and (64, 201)